In [6]:
from import_src import import_src
import_src()

Append /home/ubuntu/khiempm/MLOpsVN/src to sys.path


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px

In [8]:
link_data_test = '../../data/captured_data/phase-2/prob-2/'
uncertain_data = '../../data/captured_data/phase-2/prob-2/processed/'
train_data = '../../data/train_data/phase-2/prob-2/train_x.parquet'
y_data = '../../data/train_data/phase-2/prob-2/train_y.parquet'

In [19]:
train_x = pd.read_parquet(train_data)
train_x["label"] = pd.read_parquet(y_data)["label"]

In [20]:
captured_x = pd.DataFrame()
for file_path in os.listdir(link_data_test):
    if 'parquet' not in file_path:
        continue
    captured_data = pd.read_parquet(f'{link_data_test}{file_path}')
    captured_x = pd.concat([captured_x, captured_data])

In [21]:
feature_group = train_x.columns.tolist()
feature_group.remove("label")
dup_train = train_x.groupby(feature_group).agg(label_nunique=("label", "nunique")).reset_index()

In [23]:
dup_train[dup_train["label_nunique"] != 1]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label_nunique
296,0.000000,119,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,3.0,3.0,9.0,0.0,0.0,0.0,5.0,7.0,0.0,2
297,0.000001,0,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,1.0,1.0,5.0,0.0,0.0,0.0,2.0,5.0,0.0,3
298,0.000001,21,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,4.0,4.0,8.0,0.0,0.0,0.0,5.0,5.0,0.0,3
299,0.000001,27,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,3.0,3.0,5.0,0.0,0.0,0.0,4.0,5.0,0.0,4
300,0.000001,29,0,4,2.0,0.0,200.0,0.0,254.0,0.0,...,1.0,1.0,4.0,0.0,0.0,0.0,9.0,4.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35634,59.687565,78,0,4,106.0,0.0,28832.0,0.0,254.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3
35638,59.744881,78,0,5,68.0,0.0,18496.0,0.0,254.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3
35639,59.760056,78,0,4,180.0,0.0,48960.0,0.0,254.0,0.0,...,1.0,1.0,4.0,0.0,0.0,0.0,24.0,1.0,0.0,3
35642,59.810604,78,0,4,130.0,0.0,35360.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2


In [24]:
dup_train["feature2"].unique()

array([  6, 118, 119,   0,  21,  27,  29,  30,  32,  35,  38,  44,  46,
        49,  53,  56,  58,  66,  67,  69,  72,  75,  77,  89,  90,  92,
        95,  96,  99, 103, 109, 112, 116, 122, 123, 126, 128,   1,   3,
         9,  15,  25,  34,  52,  60,  70,  76,  84,  93, 108, 117, 124,
       125, 130,   4,   8,  10,  12,  13,  14,  17,  18,  19,  22,  23,
        24,  31,  33,  36,  39,  40,  41,  42,  43,  45,  47,  50,  51,
        54,  57,  59,  61,  62,  63,  64,  65,  68,  79,  80,  82,  83,
        85,  86,  87,  88,  94,  98, 100, 101, 102, 110, 111, 113, 114,
       127, 129, 131,   2,   5,   7,  26,  71,  81,  97, 105, 106, 115,
        20,  48,  55, 104, 121,  16,  78, 120,  74,  28,  37, 107,  11,
        73,  91])

In [25]:
dup_test = captured_x.groupby(feature_group).agg(count_num=("feature1", "count")).reset_index()

In [26]:
dup_test[dup_test["feature3"] == -1]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,count_num
0,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
1,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
2,0.000000,-1,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
9,0.000000,6,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,42
10,0.000000,6,-1,4,1.0,0.0,46.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8982,33.520821,78,-1,-1,70.0,0.0,7560.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,3.0,6.0,0.0,42
9012,47.485291,78,-1,5,70.0,0.0,19040.0,0.0,254.0,0.0,...,2.0,2.0,2.0,0.0,0.0,0.0,6.0,2.0,0.0,42
9022,51.071350,112,-1,-1,20.0,14.0,1000.0,622.0,254.0,252.0,...,3.0,3.0,5.0,0.0,0.0,0.0,4.0,6.0,0.0,42
9046,58.881458,78,-1,5,68.0,0.0,18496.0,0.0,254.0,0.0,...,2.0,2.0,5.0,0.0,0.0,0.0,4.0,2.0,0.0,42


In [27]:
merge_ = dup_train.merge(dup_test, how="inner", on=feature_group)

In [32]:
merge_[merge_["label_nunique"] != 1]["count_num"].sum()

33152